In [35]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm


In [36]:
def short_name(team):
        short_name={'Chennai Super Kings':'CSK',
    'Mumbai Indians':'MI',
    'Gujarat Titans':'GT',
    'Kolkata Knight Riders':'KKR',
    'Punjab Kings':'PBKS',
    'Sunrisers Hyderabad':'SRH',
    'Rajasthan Royals':'RR',
    'Lucknow Super Giants':'LSG',
    'Delhi Capitals':'DC',
    'Royal Challengers Bangalore':'RCB'
    }
        return short_name[team]

In [37]:
def get_moth(url):
    cricbuzz_highlights_url=url
    cricbuzz_highlights_url=cricbuzz_highlights_url.replace('cricket-full-commentary','cricket-scores')
    cricbuzz_highlights_url
    page=requests.get(cricbuzz_highlights_url)
    soup=BeautifulSoup(page.content)
    try: 
        win=soup.find('div',{'class':'cb-col cb-col-100 cb-min-stts cb-text-complete'}).text
        #cb-col cb-col-100 cb-mini-col cb-min-comp ng-scope
        #cb-link-undrln ng-binding
        a=soup.find('span',{'class':'cb-text-gray cb-font-12'})
        try:
            man_of_the_match=a.next_sibling.next_sibling.next_sibling.text
        except:
            man_of_the_match=None
    except:
        win = None
        man_of_the_match=None
    file_name=url.split('/')[-1]
    df=pd.read_csv(f'files/2021/{file_name}.csv')
    df['win']=win
    df['man_of_the_match']=man_of_the_match
    df.to_csv(f'files/2021/{file_name}.csv',index=False)





In [38]:
def find_toss(url):
    file_name=url.split('/')[-1]
    df=pd.read_csv(f'files/2023/{file_name}.csv')
    try :
        toss=df['toss'][0]
        toss_win=toss[:toss.find('have')]
        toss_choose=toss.split(' ')[-1]

        df['Toss_winner']=short_name(toss_win.strip())
        df['toss_chosen']=toss_choose
        df.to_csv(f'files/2023/{file_name}.csv',index=False)
    except:
        print('error')
        pass
#find_toss('https://www.cricbuzz.com/cricket-full-commentary/35612/mi-vs-rcb-1st-match-indian-premier-league-2021')

In [46]:

def find_innings(url):
    file_name=url.split('/')[-1]
    #print(file_name)
    df=pd.read_csv(f'files/2023/{file_name}.csv')
       
    try :
        toss_winner=df['Toss_winner'][0]
        #print(toss_winner)
        toss_choose=df['toss_chosen'][0]
        #print(toss_choose)
        if toss_choose=='field':
            #print('fielding')
           
            #print(df[df['innings']== f'{toss_winner} Inns'.strip()]['innings'])
            df['innings'].mask(df['innings']!=f'{toss_winner} Inns'.strip(),'1st Inns',inplace=True)
            df['innings'].mask(df['innings']==f'{toss_winner} Inns'.strip(),'2nd Inns',inplace=True)
            
            #print(df)
        else:
            df['innings'].mask(df['innings']!=f'{toss_winner} Inns'.strip(),'2nd Inns',inplace=True)
            df['innings'].mask(df['innings']==f'{toss_winner} Inns'.strip(),'1st Inns',inplace=True)
            
            
           
        #display(df)
        df.to_csv(f'files/2023/{file_name}.csv',index=False)
    except:
        print('error')

#find_innings('https://www.cricbuzz.com/cricket-full-commentary/66442/csk-vs-kkr-61st-match-indian-premier-league-2023')

In [80]:
def find_bowling_team(url):
    file_name=url.split('/')[-1]
    #print(file_name)
    df=pd.read_csv(f'files/2023/{file_name}.csv')
    try :
        toss_winner=df['Toss_winner'][0]
        toss_choose=df['toss_chosen'][0]
        a_team=short_name(df['team_a'][0])
        b_team=short_name(df['team_b'][0])
        teams=[a_team,b_team]
        for i in teams:
            if i==toss_winner:
                teams.remove(i)
        opt=teams[0]
        if toss_choose=='bat':
            df.loc[df['innings']=='1st Inns','batting_team']=toss_winner
            df.loc[df['innings']=='1st Inns','bowling_team']=opt
            df.loc[df['innings']=='2nd Inns','batting_team']=opt
            df.loc[df['innings']=='2nd Inns','bowling_team']=toss_winner
        if toss_choose=='field':
            df.loc[df['innings']=='1st Inns','batting_team']=opt
            df.loc[df['innings']=='1st Inns','bowling_team']=toss_winner
            df.loc[df['innings']=='2nd Inns','batting_team']=toss_winner
            df.loc[df['innings']=='2nd Inns','bowling_team']=opt
            
        #display(df)
        df.to_csv(f'files/2023/{file_name}.csv',index=False)
    except Exception as e:
        print(e)
       
find_bowling_team('https://www.cricbuzz.com/cricket-full-commentary/66519/kkr-vs-gt-39th-match-indian-premier-league-2023')

In [81]:
#gt-vs-csk-1st-match-indian-premier-league-2023
#mi-vs-rcb-1st-match-indian-premier-league-2021
links=[]
with open('match_links/2023_season_match_links.txt','r') as file:
    for line in file:
        links.append(line.strip())
print(links)
for i in tqdm(links):
    #print(i)
    find_bowling_team(i)

['https://www.cricbuzz.com/cricket-full-commentary/66169/gt-vs-csk-1st-match-indian-premier-league-2023', 'https://www.cricbuzz.com/cricket-full-commentary/66173/pbks-vs-kkr-2nd-match-indian-premier-league-2023', 'https://www.cricbuzz.com/cricket-full-commentary/66176/lsg-vs-dc-3rd-match-indian-premier-league-2023', 'https://www.cricbuzz.com/cricket-full-commentary/66183/srh-vs-rr-4th-match-indian-premier-league-2023', 'https://www.cricbuzz.com/cricket-full-commentary/66190/rcb-vs-mi-5th-match-indian-premier-league-2023', 'https://www.cricbuzz.com/cricket-full-commentary/66197/csk-vs-lsg-6th-match-indian-premier-league-2023', 'https://www.cricbuzz.com/cricket-full-commentary/66204/dc-vs-gt-7th-match-indian-premier-league-2023', 'https://www.cricbuzz.com/cricket-full-commentary/66208/rr-vs-pbks-8th-match-indian-premier-league-2023', 'https://www.cricbuzz.com/cricket-full-commentary/66211/kkr-vs-rcb-9th-match-indian-premier-league-2023', 'https://www.cricbuzz.com/cricket-full-commentary/

100%|██████████| 75/75 [00:00<00:00, 98.93it/s] 

'Toss_winner'


In [41]:
links[11]

'https://www.cricbuzz.com/cricket-full-commentary/66225/mi-vs-csk-12th-match-indian-premier-league-2023'